## Exercise 3: Designing and Implementing a Data Warehouse
In this exercise, we will design and implement a basic data warehouse using the data from our online retail store. We will understand key data warehousing concepts, design a star schema, implement it using SQL, and perform an ETL process using Python.

### Task 8: Designing a Star Schema
Description: Design a star schema for the online retail store to facilitate efficient data analysis.

**Instructions:**

1. Define the purpose and attributes of each table as outlined above.
2. Identify the grain of the fact table, which is the level of detail (e.g., one row per product sold per transaction).
3. Ensure that dimension tables contain descriptive attributes that facilitate detailed analysis.

Components of the Star Schema:

1. Fact Table:
    - Sales_Fact: Contains sales transaction data.

2. Dimension Tables:
    - Date_Dim: Contains date-related information.
    - Customer_Dim: Contains customer information.
    - Product_Dim: Contains product information.
    - Category_Dim: Contains product category information.
    - Store_Dim (Optional): Contains store or sales channel information.


Design Details:

1. Sales_Fact Table:
    - sale_id (primary key)
    - date_id (foreign key to Date_Dim)
    - customer_id (foreign key to Customer_Dim)
    - product_id (foreign key to Product_Dim)
    - quantity_sold
    - total_price

2. Date_Dim Table:
    - date_id (primary key)
    - date (full date)
    - day
    - month
    - quarter
    - year
    - weekday

3. Customer_Dim Table:
    - customer_id (primary key)
    - first_name
    - last_name
    - email
    - phone_number
    - address
    - city
    - state
    - zip_code
    - registration_date

4. Product_Dim Table:
    - product_id (primary key)
    - product_name
    - description
    - price
    - category_id (foreign key to Category_Dim)

5. Category_Dim Table:
    - category_id (primary key)
    - category_name
    - category_description
    
ER Diagram of Star Schema:
- Centered around Sales_Fact, connected to each dimension table via foreign keys.

### Task 9: Implementing the Star Schema using SQL
Description: Create the star schema in your database by defining the fact and dimension tables with appropriate constraints.

**Instructions:**

1. Create a new schema to represne the data warehouse.

In [ ]:
%% SQL

CREATE SCHEMA IF NOT EXISTS orders_dw;

2. Create dimensional tables (Date_Dim, Customer_Dim, Product_Dim, Category_Dim) with the specified attributes.

In [ ]:
%% SQL

-- Create Date_Dim Table
CREATE TABLE date_Dim (
    date_id INTEGER PRIMARY KEY,
    date DATE NOT NULL,
    day INTEGER,
    month INTEGER,
    quarter INTEGER,
    year INTEGER,
    weekday VARCHAR(10)
);

-- Create Customer_Dim Table
CREATE TABLE customer_Dim (
    customer_id INTEGER PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    phone_number VARCHAR(20),
    address VARCHAR(200),
    city VARCHAR(50),
    state VARCHAR(50),
    zip_code VARCHAR(10),
    registration_date DATE
);

-- Create Category_Dim Table
CREATE TABLE category_Dim (
    category_id INTEGER PRIMARY KEY,
    category_name VARCHAR(100),
    category_description TEXT
);

-- Create Product_Dim Table
CREATE TABLE product_Dim (
    product_id INTEGER PRIMARY KEY,
    product_name VARCHAR(100),
    description TEXT,
    price DECIMAL(10,2),
    category_id INTEGER,
    FOREIGN KEY (category_id) REFERENCES Category_Dim(category_id)
);

3. Create the fact table (Sales_Fact) with the specified attributes and foreign key constraints.

In [ ]:
%% SQL

-- Create Sales_Fact Table
CREATE TABLE sales_fact (
    sale_id SERIAL PRIMARY KEY,
    date_id INTEGER,
    customer_id INTEGER,
    product_id INTEGER,
    quantity_sold INTEGER,
    total_price DECIMAL(10,2),
    FOREIGN KEY (date_id) REFERENCES Date_Dim(date_id),
    FOREIGN KEY (customer_id) REFERENCES Customer_Dim(customer_id),
    FOREIGN KEY (product_id) REFERENCES Product_Dim(product_id)
);

4. Insert sample data into the dimension tables to populate them.

In [ ]:
%% SQL

-- Insert sample data into Date_Dim Table
INSERT INTO date_Dim (date_id, date, day, month, quarter, year, weekday)
VALUES (1, '2022-01-01', 1, 1, 1, 2022, 'Saturday'),
       (2, '2022-01-02', 2, 1, 1, 2022, 'Sunday'),
       (3, '2022-01-03', 3, 1, 1, 2022, 'Monday');

INSERT INTO category_Dim
	select *
	from public.categories 
;

INSERT INTO product_Dim
	select *
	from public.products 
;

INSERT INTO customer_Dim
	select *
	from public.customers 
;

INSERT INTO sales_fact
    SELECT 
        o.order_id,
        strftime('%Y%m%d', o.order_date) AS date_id,
        o.customer_id,
        oi.product_id,
        oi.quantity,
        oi.quantity * oi.unit_price AS total_price
    FROM public.orders o
    JOIN public.orderitems oi 
    	ON o.order_id = oi.order_id
;

### Task 10: Querying the Data Warehouse
Description: Execute analytical queries on the data warehouse to derive insights.

**Instructions:**

1. Total Sales per Product:

In [ ]:
%% SQL

SELECT 
    Product_Dim.product_name,
    SUM(Sales_Fact.quantity_sold) AS total_units_sold,
    SUM(Sales_Fact.total_price) AS total_sales
FROM Sales_Fact
JOIN Product_Dim ON Sales_Fact.product_id = Product_Dim.product_id
GROUP BY Product_Dim.product_name
ORDER BY total_sales DESC
;

2. Monthly Sales Performance:

In [ ]:
%% SQL

SELECT 
    Date_Dim.month,
    Date_Dim.year,
    SUM(Sales_Fact.total_price) AS monthly_sales
FROM Sales_Fact
JOIN Date_Dim ON Sales_Fact.date_id = Date_Dim.date_id
GROUP BY Date_Dim.year, Date_Dim.month
ORDER BY Date_Dim.year, Date_Dim.month
;

3. Top Customers by Sales:

In [ ]:
%% SQL

SELECT 
    Customer_Dim.first_name || ' ' || Customer_Dim.last_name AS customer_name,
    SUM(Sales_Fact.total_price) AS total_spent
FROM Sales_Fact
JOIN Customer_Dim ON Sales_Fact.customer_id = Customer_Dim.customer_id
GROUP BY Sales_Fact.customer_id
ORDER BY total_spent DESC
LIMIT 5
;

4. Sales by Category:

In [ ]:
%% SQL

SELECT 
    Category_Dim.category_name,
    SUM(Sales_Fact.total_price) AS total_sales
FROM Sales_Fact
JOIN Product_Dim ON Sales_Fact.product_id = Product_Dim.product_id
JOIN Category_Dim ON Product_Dim.category_id = Category_Dim.category_id
GROUP BY Category_Dim.category_name
ORDER BY total_sales DESC
;

5. Daily Sales Trends:

In [ ]:
%% SQL

SELECT 
    Date_Dim.date,
    SUM(Sales_Fact.total_price) AS daily_sales
FROM Sales_Fact
JOIN Date_Dim ON Sales_Fact.date_id = Date_Dim.date_id
GROUP BY Date_Dim.date
ORDER BY Date_Dim.date
;